# Asymptotic expansion of the energy

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\order}[2][1]{#2^{(#1)}}\newcommand{\reals}{\mathbb{R}}$ custom macros.

In [1]:
import json
import lsk

from sympy import *

with open("setting-up_the_computational_stage.out.json", "r") as f:
    data = json.load(f)

We first define the asymptotic expansion of $\lambda$ w.r.t. the (unspecified) parameter $\eta$.

In [11]:
η, λ1, λ2, λ3, λ4 = symbols(r"\eta {\order[1]{\lambda}} {\order[2]{\lambda}} {\order[3]{\lambda}} {\order[4]{\lambda}}")
λ = λ1 * η + λ2 * η**2 / 2 + λ3 * η**3 / 6 + λ4 * η**4 / 24

In [12]:
#| code-fold: true
lsk.display_latex_equation(r"\lambda(\eta)", λ)

<IPython.core.display.Math object>

The fundamental branch $u^\ast(\lambda)$ was previously defined as an asymptotic expansion w.r.t. $\lambda$.

In [13]:
u_star_of_λ = eval(data["u_star"])

In [14]:
#| code-fold: true
lsk.display_latex_equation(r"u^\ast(\lambda)", u_star_of_λ)

<IPython.core.display.Math object>

 We now define the fundamental branch as an asymptotic expansion w.r.t. $\eta$, keeping only fourth-order terms.

In [17]:
u_star = u_star_of_λ.subs(Symbol("\lambda"), λ + O(η**5)).expand().removeO()

In [18]:
#| code-fold: true
lsk.display_latex_equation(r"u^\ast(\eta)", u_star)

<IPython.core.display.Math object>

In [19]:
E_ = eval(data["E"])

In [23]:
u = (u_star
     + η * Symbol(r"\xi_i^{(1)}v_i") 
     + η**2 / 2 * (Symbol(r"\xi_j^{(2)}v_j")
                   + Symbol(r"\xi_k^{(1)}\xi_l^{(1)}w_{kl}")
                   + 2 * λ1 * Symbol(r"\xi_m^{(1)}w_{m\lambda}"))
     + η**3 / 6 * Symbol(r"{\order[3]{u}}")
     + η**4 / 24 * Symbol(r"{\order[4]{u}}")).expand()

In [24]:
#| code-fold: true
lsk.display_latex_equation(r"u(\eta)", u)

<IPython.core.display.Math object>

In [25]:
E_

-\E_2*\dddot{u}_0*\lambda**3*u/6 - \E_2*\ddot{u}_0*\lambda**2*u/2 - \E_2*\dot{u}_0*\lambda*u + \E_2*u**2/2 + \E_3*\ddot{u}_0*\dot{u}_0*\lambda**3*u/2 - \E_3*\ddot{u}_0*\lambda**2*u**2/4 + \E_3*\dot{u}_0**2*\lambda**2*u/2 - \E_3*\dot{u}_0*\lambda*u**2/2 + \E_3*u**3/6 - \E_4*\dot{u}_0**3*\lambda**3*u/6 + \E_4*\dot{u}_0**2*\lambda**2*u**2/4 - \E_4*\dot{u}_0*\lambda*u**3/6 + \E_4*u**4/24 + \E_{\lambda\lambda\lambda\lambda}*\lambda**4/24 + \E_{\lambda\lambda\lambda}*\lambda**3/6 + \E_{\lambda\lambda}*\lambda**2/2 + \E_{\lambda}*\lambda - \ddot{\E}_2*\dot{u}_0*\lambda**3*u/2 + \ddot{\E}_2*\lambda**2*u**2/4 - \ddot{u}_0*\dot{\E}_2*\lambda**3*u/2 - \dot{\E}_2*\dot{u}_0*\lambda**2*u + \dot{\E}_2*\lambda*u**2/2 + \dot{\E}_3*\dot{u}_0**2*\lambda**3*u/2 - \dot{\E}_3*\dot{u}_0*\lambda**2*u**2/2 + \dot{\E}_3*\lambda*u**3/6

In [26]:
E_star = (E_.subs({Symbol("\lambda"): λ,
                   Symbol("u"): u_star}).expand() + O(η**5)).removeO()

In [27]:
E_star

-\E_2*\dddot{u}_0*\dot{u}_0*\eta**4*{\order[1]{\lambda}}**4/6 - \E_2*\ddot{u}_0**2*\eta**4*{\order[1]{\lambda}}**4/8 - 3*\E_2*\ddot{u}_0*\dot{u}_0*\eta**4*{\order[1]{\lambda}}**2*{\order[2]{\lambda}}/4 - \E_2*\ddot{u}_0*\dot{u}_0*\eta**3*{\order[1]{\lambda}}**3/2 - \E_2*\dot{u}_0**2*\eta**4*{\order[1]{\lambda}}*{\order[3]{\lambda}}/6 - \E_2*\dot{u}_0**2*\eta**4*{\order[2]{\lambda}}**2/8 - \E_2*\dot{u}_0**2*\eta**3*{\order[1]{\lambda}}*{\order[2]{\lambda}}/2 - \E_2*\dot{u}_0**2*\eta**2*{\order[1]{\lambda}}**2/2 + \E_3*\ddot{u}_0*\dot{u}_0**2*\eta**4*{\order[1]{\lambda}}**4/4 + \E_3*\dot{u}_0**3*\eta**4*{\order[1]{\lambda}}**2*{\order[2]{\lambda}}/4 + \E_3*\dot{u}_0**3*\eta**3*{\order[1]{\lambda}}**3/6 - \E_4*\dot{u}_0**4*\eta**4*{\order[1]{\lambda}}**4/24 + \E_{\lambda\lambda\lambda\lambda}*\eta**4*{\order[1]{\lambda}}**4/24 + \E_{\lambda\lambda\lambda}*\eta**4*{\order[1]{\lambda}}**2*{\order[2]{\lambda}}/4 + \E_{\lambda\lambda\lambda}*\eta**3*{\order[1]{\lambda}}**3/6 + \E_{\lambda\lam

In [28]:
E = (E_.subs({Symbol("\lambda"): λ,
              Symbol("u"): u}).expand() + O(η**5)).removeO()

In [29]:
E

-\E_2*\dddot{u}_0*\dot{u}_0*\eta**4*{\order[1]{\lambda}}**4/6 - \E_2*\ddot{u}_0**2*\eta**4*{\order[1]{\lambda}}**4/8 - 3*\E_2*\ddot{u}_0*\dot{u}_0*\eta**4*{\order[1]{\lambda}}**2*{\order[2]{\lambda}}/4 - \E_2*\ddot{u}_0*\dot{u}_0*\eta**3*{\order[1]{\lambda}}**3/2 - \E_2*\dot{u}_0**2*\eta**4*{\order[1]{\lambda}}*{\order[3]{\lambda}}/6 - \E_2*\dot{u}_0**2*\eta**4*{\order[2]{\lambda}}**2/8 - \E_2*\dot{u}_0**2*\eta**3*{\order[1]{\lambda}}*{\order[2]{\lambda}}/2 - \E_2*\dot{u}_0**2*\eta**2*{\order[1]{\lambda}}**2/2 + \E_2*\eta**4*\xi_i^{(1)}v_i*{\order[3]{u}}/6 + \E_2*\eta**4*\xi_j^{(2)}v_j**2/8 + \E_2*\eta**4*\xi_j^{(2)}v_j*\xi_k^{(1)}\xi_l^{(1)}w_{kl}/4 + \E_2*\eta**4*\xi_j^{(2)}v_j*\xi_m^{(1)}w_{m\lambda}*{\order[1]{\lambda}}/2 + \E_2*\eta**4*\xi_k^{(1)}\xi_l^{(1)}w_{kl}**2/8 + \E_2*\eta**4*\xi_k^{(1)}\xi_l^{(1)}w_{kl}*\xi_m^{(1)}w_{m\lambda}*{\order[1]{\lambda}}/2 + \E_2*\eta**4*\xi_m^{(1)}w_{m\lambda}**2*{\order[1]{\lambda}}**2/2 + \E_2*\eta**3*\xi_i^{(1)}v_i*\xi_j^{(2)}v_j/2 + \E_2*\e

In [30]:
ΔE = E - E_star

The terms of order 0 and 1 in $\eta$ are identically null:

In [39]:
assert ΔE.coeff(η, 0) == 0
assert ΔE.coeff(η, 1) == 0

The second-order term reads

In [44]:
ΔE2 = ΔE.coeff(η, 2)

In [49]:
#| code-fold: true
lsk.display_latex_equation(r"\order[2]{\Delta\E}", ΔE2)

<IPython.core.display.Math object>

The above expression should be understood as $\frac{1}{2} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_2(v_i, v_j)$, which vanishes since $v_i$ and $v_j$ belong to the kernel of $\E_2$. We now turn to the third-order term

In [46]:
ΔE3 = ΔE.coeff(η, 3)

In [50]:
#| code-fold: true
lsk.display_latex_equation(r"\order[3]{\Delta\E}", ΔE3)

<IPython.core.display.Math object>

Observing that $\E_2(v_i, \bullet) = 0$ and using Eqs. (@eq-20230125062505), (@eq-20230125062510) and (@eq-20230125062336), the above expression reads reduces to

$$
\begin{aligned}[b]
\order[3]{\Delta\E}
&= \tfrac{1}{6} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \E_3(v_i, v_j, v_k) + \tfrac{1}{2} \, \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, v_j)\\
&= \tfrac{1}{6} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, E_{ijk} + \tfrac{1}{2} \, \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{E}_{ij}\\
&= \tfrac{1}{6} \, \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{E}_{ij}.
\end{aligned}
$$

Finally, the fourth-order term is simplified.

In [51]:
ΔE4 = ΔE.coeff(η, 4)

In [52]:
#| code-fold: true
lsk.display_latex_equation(r"\order[4]{\Delta\E}", ΔE4)

<IPython.core.display.Math object>

Upon simplification of $\E_2(v_i, \bullet) = 0$, the above expression should be understood as follows

$$
\begin{aligned}
\order[4]{\Delta\E} ={}& \tfrac{1}{8} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_2(w_{ij}, w_{kl})
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \E_2(w_{ij}, w_{k\lambda})\\
&+ \tfrac{1}{2} \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_2(w_{i\lambda}, w_{j\lambda})
+ \tfrac{1}{4} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[2]{\xi}_k \, \E_3(v_i, v_j, v_k)\\
&+ \tfrac{1}{4} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_3(v_i, v_j, w_{kl})
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \E_3(v_i, v_j, w_{k\lambda})\\
&+ \tfrac{1}{24} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_4(v_i, v_j, v_k, v_l)
+ \tfrac{1}{4} \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \ddot{\E}_2(v_i, v_j)\\
&+ \tfrac{1}{4} \order[2]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, v_j)
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[2]{\xi}_j \, \dot{\E}_2(v_i, v_j)\\
&+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \dot{E}_2(v_i, w_{jk})
+ \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, w_{j\lambda})\\
&+ \tfrac{1}{6} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \dot{\E}_3(v_i, v_j, v_k),
\end{aligned}
$$ {#eq-20230402102020}

which simplifies (see proof in @sec-20230402101835)

$$
\begin{aligned}
\order[4]{\Delta\E} ={} & \tfrac{1}{24} \bigl\{ E_{ijkl} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l + 4\order[1]{\lambda} \, \dot{E}_{ijk} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k\\
&+ 6 \bigl[ \bigl( \order[1]{\lambda} \bigr)^2 \, \ddot{E}_{ij} + \order[2]{\lambda} \, \dot{E}_{ij} \bigr] \, \order[1]{\xi}_i \, \order[1]{\xi}_j \bigr\}.
\end{aligned}
$$ {#eq-20230402101651}